Last update: Apr 14

# INSTRUCTIONS



1.   Upload the ".json" file (from ABSApp output)
2.   In the top menu bar, click "Runtime" -> "Run all"
3.   Wait for the script to finish
4.   Refresh the left panel
5.   Download the output file
6.   Open the "SQL_inserts.txt" file
7.   Copy the SQL insert statements into MySQL Workbench
8.   Use "Ctrl + A" to select all, and run all by clicking "⚡️"
9.   Wait until your database is populated






Import

In [0]:
import json, re, datetime
from datetime import datetime
import pandas as pd
import numpy as np

Helper functions for generting SQL insert statements

In [0]:
path_to_ASBA_output = '/content/drive/My Drive/NLP Preferences/forum_data_and_scrapers/ABSApp Output Data/charles_data_mar26.json'
path_to_treatment_names = '/content/drive/My Drive/NLP Preferences/database_related/populate_database/approved_drug_names.csv'

In [0]:
# INPUT: string ; author's flair
# OUTPUT: tuple ; (gender, age)
def parse_flair(flair):
  # NOTE:
  # - idea: use disease/drug names when parsing their flair
  # FOR REDDIT r/MultipleSclerosis
  flair = " " + flair + " "
  # gender
  gender = re.findall("[^A-Za-z]+([MFmf])[^A-Za-z]+", flair)
  if len(gender):
    gender = gender[0]
  else:
    gender = "unknown"
  # age
  age = re.findall("([0-9]{2})[MFmf]|[MFmf]([0-9]{2})|[^0-9']+([2-9][0-9])[^0-9]+", flair) # assume all users are over age 19
  if age:
    for t in age[0]:
      if t:
        age = t
        break
  else:
    age = 0
  diagnosis_date = re.findall("Dx.*[0-9]{4} | Dx.*[1-2][0-9].[1-2][0-9]",flair)
  return (gender, age)



# INPUT: string ; text | int ; index of aspect | string ; mode of detection
# OUTPUT: array list ; treatments in DrugBank that occured in this string
def detect_treatment(txt, aspect_pos, mode):
    res = ""
    treatment_name_list = []
    treatment_df = pd.read_csv(path_to_treatment_names, header=None)
      for treatment_name in treatment_df[treatment_df.columns[0]]:
          treatment_name_list.append(treatment_name)
    if mode not in ['thread title','same sentence','same post','1 sentence-length before','2 sentence-length before']:
      raise Exception('{} is not a valid mode for detect_treatment'.format(mode))
    treatment_and_pos = {} # format is {treatment_name: position in text}
    for t in ['ocrevus', 'copaxone', 'tysabri', 'tecfidera', 'gilenya', 'aubagio', 'baclofen','glatiramer']: # TODO: populate list of drugs using DrugBank.ca data
      match = re.search(t, txt.lower(), re.IGNORECASE) # TODO: allows multiple treatments in describes relation
      if match != None:
        treatment_and_pos[t.title()] = match.span()[1]
      # TODO: refactor duplicate logic
    if len(treatment_and_pos) == 0:
      return ""
    if mode in ['thread title','same sentence','same post']:
      if len(treatment_and_pos) > 1:
        print('detected more than 1 treatments for {} and the mode was {}'.format(txt, mode))
      # return the last treatment name before aspect index
    closest_treatment_and_dictance_to_aspect = ("",100000)
    for name, treatment_pos in treatment_and_pos.items():
      if treatment_pos > aspect_pos and mode in ['1 sentence-length before','2 sentence-length before']: continue # ignore this treatment if it occured after the aspect for certain modes
      if abs(aspect_pos - treatment_pos) < closest_treatment_and_dictance_to_aspect[1]:
        closest_treatment_and_dictance_to_aspect = (name, aspect_pos - treatment_pos)
        res = name
    return res


# INPUT: string ; text.   string ; forum name
# OUTPUT: the detected disease for forum
def detect_disease(body, forum):
  forum_disease_dict = {
      'r/MultipleSclerosis' : 'multiple sclerosis'
  }
  return forum_disease_dict[forum]

# INPUT: string; a row of data
# OUTPUT: string ; author SQL insert
def gen_author(row):
  res = ''
  name = row['author']
  forum_name = row['forum']
  flair = str(row['flair'])
  gender, age = parse_flair(str(flair))
  res = res + 'INSERT IGNORE INTO author (author_username, author_gender, author_age, author_flair, author_profile_url, author_forum_name) VALUES (\"' + name + '\", \"'+ gender + '\", '+ str(age) + ',\"'+ flair +'\","' + 'http://reddit.com/user/'+ name + '","{}");'.format(forum_name)+  '\n'
  return res



# OUTPUT: string ; disease SQL insert
# NOTE:   needs to be hand-coded
def gen_disease():
  # following statistics from https://subredditstats.com/r/multiplesclerosis
  res = 'INSERT IGNORE INTO disease (disease_name) VALUES ("multiple sclerosis");'
  return res


# OUTPUT: string ; forum SQL insert
# NOTE: needs to be hand-coded
def gen_forum():
  # following statistics from https://subredditstats.com/r/multiplesclerosis
  res = """INSERT IGNORE INTO forum (forum_name, forum_url, forum_disease_name, forum_user_count, forum_activity_count) VALUES ('r/MultipleSclerosis', 'https://www.reddit.com/r/MultipleSclerosis/', 'multiple sclerosis', 15537, 23504);\n"""
  return res

def gen_category():
  res = "INSERT IGNORE INTO category (category_name) VALUES ('temporary category');"
  return res


# INPUT:  string ; a row of data
# OUTPUT: string ; post SQL insert statement
def gen_post(row):
  res = ''
  post_id = row['id']
  body = row['body']
  body = re.sub('"',"'",body)
  timestamp = int(row['time'])
  timestamp = datetime.utcfromtimestamp(timestamp).strftime('%Y-%m-%d %H:%M:%S') 
  score = row['score']
  author = row['author']
  forum_name = row['forum']
  disease_name = row['disease']
  sub_title = re.sub('"',"'",row['sub_title'])
  reply_to_id = row['reply_to_id']
  size = len(body)
  res = res + 'INSERT IGNORE INTO post (post_id, post_size, post_timestamp, post_body_hash, post_forum_name, post_disease_name, post_score, post_author_username, post_thread_title, post_body, post_reply_to_post_id) VALUES (\"'  \
  + str(post_id) + '",' \
  + str(size) + ',"' \
  + str(timestamp) + '\", MD5(\"' \
  + body + '\")' \
  + ', "{}"'.format(forum_name)  \
  + ', "{}"'.format(disease_name) \
  + ',' + str(score) \
  + ',' + '"{}"'.format(author)\
  + ',"' + sub_title + '"' \
  + ',"' + body \
  + '",' + reply_to_id \
  + ');' + "\n" 
  return res

    


# INPUT: string ; a row of data
# OUTPUT: string ; sentence SQL insert statement
def gen_sentence(row):
  res = ''
  try: 
    sentences = eval(row['inference'])['_sentences']
  except:
    return ""
  timestamp = int(row['time'])
  timestamp = datetime.utcfromtimestamp(timestamp).strftime('%Y-%m-%d %H:%M:%S') 
  body = row['body']
  body = re.sub('"',"'",body)
  for s in sentences:
      sent = body[s['_start']:s['_end']]
      temp = 'INSERT IGNORE INTO sentence(sentence_body, sentence_post_id, sentence_body_hash) VALUES (\"' + sent + '\"' + ', (SELECT post_id FROM post WHERE post_body_hash = MD5(\"' + body +'\") and post_timestamp = "{}"), MD5(\"'.format(str(timestamp)) + sent +'\"));'
      res = res + temp + "\n \n"
  return res



# OUTPUT: string ; treatment SQL insert statement
def gen_treatment():
  res = ""
  treatment_df = pd.read_csv(path_to_treatment_names, header=None)
  for treatment_name in treatment_df[treatment_df.columns[0]]:
    res += 'INSERT IGNORE INTO treatment(treatment_name) VALUES ("{}");\n'.format(treatment_name)
  return res + 'INSERT IGNORE INTO treatment(treatment_name) VALUES ("");\n'
#   return """INSERT IGNORE INTO treatment(treatment_name) VALUES ("Ocrevus");\n
# INSERT IGNORE INTO treatment(treatment_name) VALUES ("Copaxone");\n
# INSERT IGNORE INTO treatment(treatment_name) VALUES ("Tysabri");\n
# INSERT IGNORE INTO treatment(treatment_name) VALUES ("Tecfidera");\n
# INSERT IGNORE INTO treatment(treatment_name) VALUES ("Gilenya");\n
# INSERT IGNORE INTO treatment(treatment_name) VALUES ("Aubagio");\n
# INSERT IGNORE INTO treatment(treatment_name) VALUES ("Baclofen");\n
# INSERT IGNORE INTO treatment(treatment_name) VALUES ("Glatiramer");\n
# INSERT IGNORE INTO treatment(treatment_name) VALUES ("None");\n"""


# INPUT: string ; a row of data
# OUTPUT: array list of tuples ; describes insert statements
def gen_describes(row):
  res = []
  try: 
      sentences = eval(row['inference'])['_sentences']
  except:
    return ""
  body = row['body']
  body = re.sub('"',"'",body)
  post_id = row['id']
  forum_name = row['forum']
  author_username = row['author']
  time = int(row['time'])
  time = datetime.utcfromtimestamp(time).strftime('%Y-%m-%d %H:%M:%S') 
  treatment_thread_title = detect_treatment(row['sub_title'],0,'thread title')
  for i,s in enumerate(sentences):
      sentence_body = body[s['_start']:s['_end']]
      # Get treatment field
      for e in s['_events']:
        aspect_location = e[0]['_start']
        same_sentence_treatment = detect_treatment(sentence_body, aspect_location - s['_start'],'same sentence') # second argument is the relative position of aspect to sentence
        same_post_treatment = detect_treatment(body, aspect_location, 'same post')
        treatment_within_110_char_upwards = detect_treatment(body[max(0,int(e[0]['_start'])-110):e[0]['_start']], aspect_location - max(0,int(e[0]['_start'])-110), '1 sentence-length before')
        treatment_within_220_char_upwards = detect_treatment(body[max(0,int(e[0]['_start'])-220):e[0]['_start']], aspect_location - max(0,int(e[0]['_start'])-220), '2 sentence-length before')
        for word in e:
          # Get aspect field
          if word['_type'] == "ASPECT":
            aspect = word['_text']
            aspect = re.sub('"',"'",aspect)
          # Get opinion field
          if word['_type'] == "OPINION":
            opi = word['_text']
            opi = re.sub('"',"'",opi.strip())
            score = word['_score']
            opi_start = word['_start']
            opi_end = opi_start + word['_len']
        temp_opi = 'INSERT IGNORE INTO opinion_word(opinion_word_name, opinion_word_polarity) values (\"' + opi.strip() + '\", ' + str(score) + ');'
        temp_aspect = 'INSERT IGNORE INTO aspect(aspect_name, aspect_category_name) values (\"' + aspect + '\","temporary category");'
        temp_describes = 'INSERT IGNORE INTO describes(describes_aspect_name, describes_opinion_word_name, describes_sentence_id, describes_same_sentence_treatment_name, describes_same_post_treatment_name, describes_110_characters_treatment_name, describes_220_characters_treatment_name, describes_thread_title_treatment_name, describes_forum_name, describes_author_username, describes_timestamp) VALUES (' \
        + '"{}"'.format(aspect) + ', ' \
        + '"{}"'.format(opi.strip()) + ',' \
        + '(SELECT sentence_id FROM sentence WHERE sentence_body_hash = MD5(\"' \
        + sentence_body + '\")),'.format(post_id)\
        + "'{}','{}', '{}', '{}','{}','{}','{}','{}'".format(same_sentence_treatment, same_post_treatment,treatment_within_110_char_upwards,treatment_within_220_char_upwards, treatment_thread_title, forum_name, author_username, time) + ');' 
        res.append((temp_opi,temp_aspect,temp_describes))
  return res




Load data

In [0]:
# OUTPUT: pd.DataFrame ; stores JSON data in dataframe, simplifies key
def load_data(filepath):
  with open(filepath, 'r+') as file:
      content = file.read()
      file.seek(0)
      content.replace('"', "'")
      file.write(content)

  with open(filepath, 'r') as fp:
    data = json.load(fp)

  # column names in new dataframe
  column_headers = [
             'id',                  # this post's ID
             'author',              # author username
             'flair',               # author flair
             'body',                # post/comment content text
             'inference',           # inference for body text
             'time',                # timestamp
             'score',               # score
             'thread_id',           # generated thread_id
             'position_in_thread',  # position in thread, 1 for post, 2 for first comment in thread, etc
             'is_post?',            # True if post, False if comment
             'forum',               # forum name
             'disease',             # disease name
             'sub_title',           # thread title
             'reply_to_id'          # the post id that this post replies to
             ]
  # data placeholder for dataframe
  flat_data = []
  # remove emojis?
  # emoji_pattern = re.compile("["
  #       u"\U0001F600-\U0001F64F"  # emoticons
  #       u"\U0001F300-\U0001F5FF"  # symbols & pictographs
  #       u"\U0001F680-\U0001F6FF"  # transport & map symbols
  #       u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
  #                          "]+", flags=re.UNICODE)
  for i, sub in enumerate(data):
    sub_row = [] # current row
    sub_row.append(sub['sub_id'])
    sub_row.append(sub['sub_author'])
    sub_row.append(sub['sub_author_flair'])
    sub_row.append(sub['sub_body'])
    try:
      sub_row.append(sub['sub_body_inference'])
    except KeyError:
      sub_row.append({})
    sub_row.append(sub['sub_time'])
    sub_row.append(sub['sub_score'])
    sub_row.append(i)
    sub_row.append(1)
    sub_row.append(True)
    sub_row.append('r/MultipleSclerosis') # TODO Change this to be a mutable variable
    sub_row.append(detect_disease(sub['sub_body'],'r/MultipleSclerosis')) # TODO change second parameter of detect_disease, forum
    sub_row.append(sub['sub_title'])
    sub_row.append("Null")
    flat_data.append(sub_row)
    assert(len(sub_row) == len(column_headers))

    # iterate through comments and add to flat_data
    for k, com in enumerate(sub['comments']):
      cur_row = []
      cur_row.append(com['comment_id'])
      cur_row.append(com['comment_author'])
      cur_row.append(com['comment_author_flair'])
      cur_row.append(com['comment_body'])
      try:
        cur_row.append(com['comment_body_inference'])
      except KeyError:
        cur_row.append({})
      cur_row.append(com['comment_time'])
      cur_row.append(com['comment_score'])
      cur_row.append(i)
      cur_row.append(k+2)
      cur_row.append(False)
      cur_row.append('r/MultipleSclerosis') # TODO Change this to be a mutable variable
      cur_row.append(detect_disease(com['comment_body'],'r/MultipleSclerosis')) # TODO change second parameter of detect_disease, forum
      cur_row.append('')
      cur_row.append('"'+com['comment_parent_id'][3:]+'"') # the JSON file contains prefix (e.g. 't3_')  for all comments
      flat_data.append(cur_row)
      assert(len(cur_row) == len(column_headers))
      
  assert(i==len(data)-1)
  df = pd.DataFrame(flat_data, columns=column_headers)
  return df

Iterate through using helpers to generate insert statements in SQL

In [0]:
def gen_all(df):
  # create dataframe for storing raw JSON data
  # create dataframe for SQL insert statements
  post_and_sentence_SQL = ""
  # iterate through 
  forum_SQL = gen_forum()
  disease_SQL = gen_disease()
  treatment_SQL = gen_treatment()
  category_SQL = gen_category()
  author_SQL = []
  aspect_SQL = []
  opinion_SQL = []
  describes_SQL = []
  for i, row in df.iterrows():
    author_SQL.append(gen_author(row))
    post_SQL = gen_post(row)
    sentence_SQL = gen_sentence(row)
    temp = gen_describes(row)
    for tupl in temp:
      aspect_SQL.append(tupl[0])
      opinion_SQL.append(tupl[1])
      describes_SQL.append(tupl[2])
    # print("{}/{} rows processed, {}% complete".format(i,len(df),round(100*i/len(df),1)))
    post_and_sentence_SQL = post_and_sentence_SQL + "\n" + post_SQL + "\n" + sentence_SQL # + "\n" + describes_SQL + "\n"

  # preserve only unique authors, aspects, and opinion_words
  author_SQL = "\n".join(list(set(author_SQL)))
  aspect_SQL = "\n".join(list(set(aspect_SQL)))
  opinion_SQL = "\n".join(list(set(opinion_SQL)))
  describes_SQL = "\n".join(describes_SQL) # do not remove describes that "look" the same
  print("-------------\nall SQL statements have been successfully generated")

  return "\n".join([disease_SQL, treatment_SQL, forum_SQL, author_SQL, category_SQL, post_SQL, post_and_sentence_SQL, aspect_SQL, opinion_SQL, describes_SQL])

Output SQL statements into text file

In [0]:
def export_SQL(all_SQL):
  file_name = r"SQL_inserts_"+str(datetime.now())[:10] + "_"+ str(datetime.now())[11:19]+ ".txt"
  SQL_file = open(file_name,'w')
  try:
    SQL_file.write(all_SQL)
    SQL_file.write("")
  except:
    print("An error occured.")
  print("""All SQL insert statements have been saved to '{}', please download it from the left panel. \n
  You might need to refresh the 'Files' panel (by clicking the button next to 'upload'), but NOT the whole page.""".format(file_name))


# Main function

In [0]:
def main():
  # data is flattened and assigned to df
  df = load_data(path_to_ASBA_output) # path to Google Drive.  For path to example data: /content/drive/My Drive/NLP/charles_example_feb14.json

  # generate insert statements
  all_SQL = "use mydb; \n" + gen_all(df) # the database name is 'mydb'


  # export file
  export_SQL(all_SQL)


main()